<a href="https://colab.research.google.com/github/musa-prog/medipack/blob/main/MediConnect_Django_Backend_Application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# mediconnect_project/manage.py
# This file is automatically generated by Django.
# You typically don't need to modify this file.

#!/usr/bin/env python
"""Django's command-line utility for administrative tasks."""
import os
import sys


def main():
    """Run administrative tasks."""
    os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'mediconnect_project.settings')
    try:
        from django.core.management import execute_from_command_line
    except ImportError as exc:
        raise ImportError(
            "Couldn't import Django. Are you sure it's installed and "
            "available on your PYTHONPATH environment variable? Did you "
            "forget to activate a virtual environment?"
        ) from exc
    execute_from_command_line(sys.argv)


if __name__ == '__main__':
    main()

```python
# mediconnect_project/mediconnect_project/settings.py
# Django settings for the MediConnect project.

import os
from pathlib import Path

# Build paths inside the project like this: BASE_DIR / 'subdir'.
BASE_DIR = Path(__file__).resolve().parent.parent


# SECURITY WARNING: keep the secret key used in production secret!
SECRET_KEY = 'django-insecure-your-secret-key-here-for-development' # Replace with a strong, random key in production

# SECURITY WARNING: don't run with debug turned on in production!
DEBUG = True

ALLOWED_HOSTS = []


# Application definition

INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'rest_framework', # Django REST Framework
    'corsheaders', # For handling CORS, important for frontend interactions
    'core', # Our custom core application
]

MIDDLEWARE = [
    'django.middleware.security.SecurityMiddleware',
    'django.contrib.sessions.middleware.SessionMiddleware',
    'corsheaders.middleware.CorsMiddleware', # CORS middleware
    'django.middleware.common.CommonMiddleware',
    'django.middleware.csrf.CsrfViewMiddleware',
    'django.contrib.auth.middleware.AuthenticationMiddleware',
    'django.contrib.messages.middleware.MessageMiddleware',
    'django.middleware.clickjacking.XFrameOptionsMiddleware',
]

ROOT_URLCONF = 'mediconnect_project.urls'

TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'DIRS': [],
        'APP_DIRS': True,
        'OPTIONS': {
            'context_processors': [
                'django.template.context_processors.debug',
                'django.template.context_processors.request',
                'django.contrib.auth.context_processors.auth',
                'django.contrib.messages.context_processors.messages',
            ],
        },
    },
]

WSGI_APPLICATION = 'mediconnect_project.wsgi.application'


# Database
# [https://docs.djangoproject.com/en/5.0/ref/settings/#databases](https://docs.djangoproject.com/en/5.0/ref/settings/#databases)

DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.postgresql',
        'NAME': 'mediconnect', # Your PostgreSQL database name
        'USER': 'your_username', # Your PostgreSQL username
        'PASSWORD': 'your_password', # Your PostgreSQL password
        'HOST': 'localhost', # Or the IP address of your PostgreSQL server
        'PORT': '5432',
    }
}


# Password validation
# [https://docs.djangoproject.com/en/5.0/ref/settings/#auth-password-validators](https://docs.djangoproject.com/en/5.0/ref/settings/#auth-password-validators)

AUTH_PASSWORD_VALIDATORS = [
    {
        'NAME': 'django.contrib.auth.password_validation.UserAttributeSimilarityValidator',
    },
    {
        'NAME': 'django.contrib.auth.password_validation.MinimumLengthValidator',
    },
    {
        'NAME': 'django.contrib.auth.password_validation.CommonPasswordValidator',
    },
    {
        'NAME': 'django.contrib.auth.password_validation.NumericPasswordValidator',
    },
]


# Internationalization
# [https://docs.djangoproject.com/en/5.0/topics/i18n/](https://docs.djangoproject.com/en/5.0/topics/i18n/)

LANGUAGE_CODE = 'en-us'

TIME_ZONE = 'UTC'

USE_I18N = True

USE_TZ = True


# Static files (CSS, JavaScript, Images)
# [https://docs.djangoproject.com/en/5.0/howto/static-files/](https://docs.djangoproject.com/en/5.0/howto/static-files/)

STATIC_URL = 'static/'

# Default primary key field type
# [https://docs.djangoproject.com/en/5.0/ref/settings/#default-auto-field](https://docs.djangoproject.com/en/5.0/ref/settings/#default-auto-field)

DEFAULT_AUTO_FIELD = 'django.db.models.BigAutoField'

# Django REST Framework settings
REST_FRAMEWORK = {
    'DEFAULT_AUTHENTICATION_CLASSES': [
        'rest_framework.authentication.SessionAuthentication',
        'rest_framework.authentication.TokenAuthentication', # For API token authentication
    ],
    'DEFAULT_PERMISSION_CLASSES': [
        'rest_framework.permissions.IsAuthenticatedOrReadOnly', # Allow read-only for unauthenticated users
    ],
    'DEFAULT_PAGINATION_CLASS': 'rest_framework.pagination.PageNumberPagination',
    'PAGE_SIZE': 10
}

# CORS settings
CORS_ALLOW_ALL_ORIGINS = True # For development, allow all origins. Restrict in production.
# CORS_ALLOWED_ORIGINS = [
#     "http://localhost:3000", # Example: if your frontend is on port 3000
# ]

```python
# mediconnect_project/mediconnect_project/urls.py
# Main URL configuration for the MediConnect project.

from django.contrib import admin
from django.urls import path, include
from rest_framework.authtoken.views import obtain_auth_token # For obtaining DRF auth tokens

urlpatterns = [
    path('admin/', admin.site.urls),
    path('api/', include('core.urls')), # Include URLs from our core app
    path('api-token-auth/', obtain_auth_token, name='api_token_auth'), # Endpoint to get auth token
]

```python
# mediconnect_project/core/models.py
# Database models for the MediConnect core application.

from django.db import models
from django.contrib.auth.models import AbstractUser

# Custom User model extending Django's AbstractUser
class User(AbstractUser):
    # Add any additional fields specific to your users (e.g., phone_number, role)
    phone_number = models.CharField(max_length=15, blank=True, null=True, unique=True)
    USER_ROLE_CHOICES = [
        ('patient', 'Patient'),
        ('pharmacy', 'Pharmacy'),
        ('logistics', 'Logistics Partner'),
        ('admin', 'Admin'),
    ]
    role = models.CharField(max_length=20, choices=USER_ROLE_CHOICES, default='patient')

    def __str__(self):
        return self.username

class Prescription(models.Model):
    user = models.ForeignKey(User, on_delete=models.CASCADE, related_name='prescriptions')
    image = models.ImageField(upload_to='prescriptions/') # Stores prescription images
    extracted_text = models.TextField(blank=True, null=True) # Text extracted by OCR
    validation_status = models.CharField(
        max_length=50,
        choices=[
            ('pending', 'Pending'),
            ('validated', 'Validated'),
            ('rejected', 'Rejected'),
        ],
        default='pending'
    )
    upload_date = models.DateTimeField(auto_now_add=True)

    def __str__(self):
        return f"Prescription for {self.user.username} ({self.id})"

class Medicine(models.Model):
    name = models.CharField(max_length=255)
    description = models.TextField(blank=True, null=True)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    stock_quantity = models.IntegerField(default=0)
    pharmacy = models.ForeignKey(User, on_delete=models.CASCADE, related_name='medicines', limit_choices_to={'role': 'pharmacy'})

    def __str__(self):
        return self.name

class Order(models.Model):
    patient = models.ForeignKey(User, on_delete=models.CASCADE, related_name='orders', limit_choices_to={'role': 'patient'})
    pharmacy = models.ForeignKey(User, on_delete=models.SET_NULL, null=True, blank=True, related_name='pharmacy_orders', limit_choices_to={'role': 'pharmacy'})
    logistics_partner = models.ForeignKey(User, on_delete=models.SET_NULL, null=True, blank=True, related_name='delivery_orders', limit_choices_to={'role': 'logistics'})
    order_date = models.DateTimeField(auto_now_add=True)
    delivery_address = models.CharField(max_length=255)
    total_amount = models.DecimalField(max_digits=10, decimal_places=2)
    ORDER_STATUS_CHOICES = [
        ('pending', 'Pending'),
        ('processing', 'Processing'),
        ('shipped', 'Shipped'),
        ('delivered', 'Delivered'),
        ('cancelled', 'Cancelled'),
    ]
    status = models.CharField(max_length=50, choices=ORDER_STATUS_CHOICES, default='pending')
    payment_status = models.CharField(
        max_length=50,
        choices=[
            ('pending', 'Pending'),
            ('paid', 'Paid'),
            ('failed', 'Failed'),
        ],
        default='pending'
    )
    # Link to prescription if the order is based on one
    prescription = models.ForeignKey(Prescription, on_delete=models.SET_NULL, null=True, blank=True, related_name='orders')

    def __str__(self):
        return f"Order {self.id} by {self.patient.username}"

class OrderItem(models.Model):
    order = models.ForeignKey(Order, on_delete=models.CASCADE, related_name='items')
    medicine = models.ForeignKey(Medicine, on_delete=models.CASCADE)
    quantity = models.IntegerField()
    price_at_order = models.DecimalField(max_digits=10, decimal_places=2) # Price at the time of order

    def __str__(self):
        return f"{self.quantity} x {self.medicine.name} for Order {self.order.id}"

```python
# mediconnect_project/core/serializers.py
# Django REST Framework serializers for our models.

from rest_framework import serializers
from .models import User, Prescription, Medicine, Order, OrderItem

class UserSerializer(serializers.ModelSerializer):
    class Meta:
        model = User
        fields = ['id', 'username', 'email', 'phone_number', 'role', 'first_name', 'last_name']
        read_only_fields = ['role'] # Role should be set by admin or during specific registration flows

class PrescriptionSerializer(serializers.ModelSerializer):
    user = serializers.ReadOnlyField(source='user.username') # Display username instead of user ID

    class Meta:
        model = Prescription
        fields = ['id', 'user', 'image', 'extracted_text', 'validation_status', 'upload_date']
        read_only_fields = ['extracted_text', 'validation_status', 'upload_date'] # These are set by backend logic

class MedicineSerializer(serializers.ModelSerializer):
    pharmacy = serializers.ReadOnlyField(source='pharmacy.username')

    class Meta:
        model = Medicine
        fields = ['id', 'name', 'description', 'price', 'stock_quantity', 'pharmacy']

class OrderItemSerializer(serializers.ModelSerializer):
    medicine_name = serializers.ReadOnlyField(source='medicine.name')

    class Meta:
        model = OrderItem
        fields = ['id', 'medicine', 'medicine_name', 'quantity', 'price_at_order']
        read_only_fields = ['price_at_order'] # Price is set automatically based on medicine price

class OrderSerializer(serializers.ModelSerializer):
    patient = serializers.ReadOnlyField(source='patient.username')
    pharmacy = serializers.ReadOnlyField(source='pharmacy.username')
    logistics_partner = serializers.ReadOnlyField(source='logistics_partner.username')
    items = OrderItemSerializer(many=True, read_only=True) # Nested serializer for order items

    class Meta:
        model = Order
        fields = [
            'id', 'patient', 'pharmacy', 'logistics_partner', 'order_date',
            'delivery_address', 'total_amount', 'status', 'payment_status',
            'prescription', 'items'
        ]
        read_only_fields = ['order_date', 'total_amount', 'status', 'payment_status'] # These are managed by backend

```python
# mediconnect_project/core/views.py
# Django REST Framework views for our API endpoints.

from rest_framework import viewsets, status
from rest_framework.decorators import action
from rest_framework.response import Response
from rest_framework.permissions import IsAuthenticated, AllowAny
from .models import User, Prescription, Medicine, Order, OrderItem
from .serializers import UserSerializer, PrescriptionSerializer, MedicineSerializer, OrderSerializer, OrderItemSerializer
from .integrations import send_sms, initiate_payment, extract_text_from_image, geocode_address
from django.db import transaction

class UserViewSet(viewsets.ModelViewSet):
    queryset = User.objects.all()
    serializer_class = UserSerializer
    permission_classes = [AllowAny] # Allow anyone to create users for registration purposes

    def get_permissions(self):
        # Allow any user to create (register)
        if self.action == 'create':
            return [AllowAny()]
        # For other actions, require authentication
        return [IsAuthenticated()]

    # Example of a custom action for a user to update their own profile
    @action(detail=False, methods=['get', 'put'], permission_classes=[IsAuthenticated])
    def me(self, request):
        if request.method == 'GET':
            serializer = self.get_serializer(request.user)
            return Response(serializer.data)
        elif request.method == 'PUT':
            serializer = self.get_serializer(request.user, data=request.data, partial=True)
            serializer.is_valid(raise_exception=True)
            serializer.save()
            return Response(serializer.data)

class PrescriptionViewSet(viewsets.ModelViewSet):
    queryset = Prescription.objects.all()
    serializer_class = PrescriptionSerializer
    permission_classes = [IsAuthenticated]

    def perform_create(self, serializer):
        # Automatically set the user for the prescription
        prescription = serializer.save(user=self.request.user)
        # In a real scenario, you would trigger OCR processing here
        # For now, we'll just simulate it.
        # extracted_text = extract_text_from_image(prescription.image.path)
        # prescription.extracted_text = extracted_text
        # prescription.save()
        print(f"Prescription {prescription.id} uploaded by {self.request.user.username}. OCR processing simulated.")

    # Custom action to trigger OCR (can be called by an admin or a background task)
    @action(detail=True, methods=['post'], permission_classes=[IsAuthenticated])
    def process_ocr(self, request, pk=None):
        prescription = self.get_object()
        try:
            # Simulate OCR processing
            extracted_text = extract_text_from_image(prescription.image.path)
            prescription.extracted_text = extracted_text
            prescription.validation_status = 'pending' # After OCR, it needs validation
            prescription.save()
            return Response({'status': 'OCR processing initiated and text extracted'}, status=status.HTTP_200_OK)
        except Exception as e:
            return Response({'error': str(e)}, status=status.HTTP_400_BAD_REQUEST)

    # Custom action to validate a prescription (typically by a pharmacy or admin)
    @action(detail=True, methods=['post'], permission_classes=[IsAuthenticated])
    def validate(self, request, pk=None):
        prescription = self.get_object()
        # Only allow pharmacies or admins to validate
        if not request.user.role in ['pharmacy', 'admin']:
            return Response({'detail': 'You do not have permission to perform this action.'}, status=status.HTTP_403_FORBIDDEN)

        validation_status = request.data.get('validation_status')
        if validation_status not in ['validated', 'rejected']:
            return Response({'detail': 'Invalid validation status. Must be "validated" or "rejected".'}, status=status.HTTP_400_BAD_REQUEST)

        prescription.validation_status = validation_status
        prescription.save()
        return Response({'status': f'Prescription marked as {validation_status}'}, status=status.HTTP_200_OK)


class MedicineViewSet(viewsets.ModelViewSet):
    queryset = Medicine.objects.all()
    serializer_class = MedicineSerializer
    permission_classes = [IsAuthenticated]

    def get_queryset(self):
        # Pharmacies can only see/manage their own medicines
        if self.request.user.role == 'pharmacy':
            return Medicine.objects.filter(pharmacy=self.request.user)
        # Admins can see all medicines
        elif self.request.user.is_staff:
            return Medicine.objects.all()
        # Patients can see all available medicines
        return Medicine.objects.all()

    def perform_create(self, serializer):
        # Only pharmacies can add medicines
        if self.request.user.role != 'pharmacy':
            raise serializers.ValidationError("Only pharmacies can add medicines.")
        serializer.save(pharmacy=self.request.user)

    def perform_update(self, serializer):
        # Only pharmacies can update their own medicines
        if self.request.user.role != 'pharmacy' or serializer.instance.pharmacy != self.request.user:
            raise serializers.ValidationError("You do not have permission to update this medicine.")
        serializer.save()

    def perform_destroy(self, instance):
        # Only pharmacies can delete their own medicines
        if self.request.user.role != 'pharmacy' or instance.pharmacy != self.request.user:
            raise serializers.ValidationError("You do not have permission to delete this medicine.")
        instance.delete()


class OrderViewSet(viewsets.ModelViewSet):
    queryset = Order.objects.all()
    serializer_class = OrderSerializer
    permission_classes = [IsAuthenticated]

    def get_queryset(self):
        # Patients can only see their own orders
        if self.request.user.role == 'patient':
            return Order.objects.filter(patient=self.request.user)
        # Pharmacies can see orders assigned to them
        elif self.request.user.role == 'pharmacy':
            return Order.objects.filter(pharmacy=self.request.user)
        # Logistics partners can see orders assigned to them
        elif self.request.user.role == 'logistics':
            return Order.objects.filter(logistics_partner=self.request.user)
        # Admins can see all orders
        elif self.request.user.is_staff:
            return Order.objects.all()
        return Order.objects.none() # Default for other roles

    def perform_create(self, serializer):
        # Only patients can create orders
        if self.request.user.role != 'patient':
            raise serializers.ValidationError("Only patients can create orders.")

        # Ensure order items are provided in the request data
        items_data = self.request.data.get('items')
        if not items_data:
            raise serializers.ValidationError({"items": "Order must contain at least one item."})

        total_amount = 0
        order_items = []

        with transaction.atomic():
            order = serializer.save(patient=self.request.user, status='pending', payment_status='pending')

            for item_data in items_data:
                medicine_id = item_data.get('medicine')
                quantity = item_data.get('quantity')

                if not medicine_id or not quantity:
                    raise serializers.ValidationError("Each item must have 'medicine' ID and 'quantity'.")

                try:
                    medicine = Medicine.objects.get(id=medicine_id)
                except Medicine.DoesNotExist:
                    raise serializers.ValidationError(f"Medicine with ID {medicine_id} does not exist.")

                if medicine.stock_quantity < quantity:
                    raise serializers.ValidationError(f"Not enough stock for {medicine.name}. Available: {medicine.stock_quantity}")

                price_at_order = medicine.price
                total_amount += price_at_order * quantity

                # Decrease stock
                medicine.stock_quantity -= quantity
                medicine.save()

                order_item = OrderItem.objects.create(
                    order=order,
                    medicine=medicine,
                    quantity=quantity,
                    price_at_order=price_at_order
                )
                order_items.append(order_item)

            order.total_amount = total_amount
            order.save()

            # Simulate payment initiation
            # if self.request.user.phone_number:
            #     payment_response = initiate_payment(self.request.user.phone_number, total_amount)
            #     print(f"M-Pesa payment initiated for order {order.id}: {payment_response}")
            # else:
            #     print(f"No phone number for user {self.request.user.username} to initiate M-Pesa payment.")

    @action(detail=True, methods=['post'], permission_classes=[IsAuthenticated])
    def update_status(self, request, pk=None):
        order = self.get_object()
        new_status = request.data.get('status')

        valid_statuses = [choice[0] for choice in Order.ORDER_STATUS_CHOICES]
        if new_status not in valid_statuses:
            return Response({'detail': 'Invalid status provided.'}, status=status.HTTP_400_BAD_REQUEST)

        # Logic for who can update which status
        if request.user.role == 'pharmacy' and order.pharmacy == request.user:
            if new_status in ['processing', 'shipped', 'cancelled']:
                order.status = new_status
                order.save()
                # Simulate SMS notification
                # send_sms(order.patient.phone_number, f"Your order {order.id} status updated to {new_status}.")
                return Response({'status': f'Order status updated to {new_status}'}, status=status.HTTP_200_OK)
            else:
                return Response({'detail': 'Pharmacies can only set status to processing, shipped, or cancelled.'}, status=status.HTTP_403_FORBIDDEN)
        elif request.user.role == 'logistics' and order.logistics_partner == request.user:
            if new_status in ['shipped', 'delivered']:
                order.status = new_status
                order.save()
                # send_sms(order.patient.phone_number, f"Your order {order.id} status updated to {new_status}.")
                return Response({'status': f'Order status updated to {new_status}'}, status=status.HTTP_200_OK)
            else:
                return Response({'detail': 'Logistics partners can only set status to shipped or delivered.'}, status=status.HTTP_403_FORBIDDEN)
        elif request.user.is_staff: # Admins can update any status
            order.status = new_status
            order.save()
            return Response({'status': f'Order status updated to {new_status}'}, status=status.HTTP_200_OK)
        else:
            return Response({'detail': 'You do not have permission to update this order status.'}, status=status.HTTP_403_FORBIDDEN)

    @action(detail=True, methods=['post'], permission_classes=[IsAuthenticated])
    def assign_pharmacy(self, request, pk=None):
        order = self.get_object()
        pharmacy_id = request.data.get('pharmacy_id')

        if not request.user.is_staff: # Only admins can assign pharmacies
            return Response({'detail': 'You do not have permission to perform this action.'}, status=status.HTTP_403_FORBIDDEN)

        try:
            pharmacy = User.objects.get(id=pharmacy_id, role='pharmacy')
            order.pharmacy = pharmacy
            order.save()
            return Response({'status': f'Order assigned to pharmacy {pharmacy.username}'}, status=status.HTTP_200_OK)
        except User.DoesNotExist:
            return Response({'detail': 'Pharmacy not found.'}, status=status.HTTP_404_NOT_FOUND)

    @action(detail=True, methods=['post'], permission_classes=[IsAuthenticated])
    def assign_logistics(self, request, pk=None):
        order = self.get_object()
        logistics_partner_id = request.data.get('logistics_partner_id')

        if not request.user.is_staff: # Only admins can assign logistics
            return Response({'detail': 'You do not have permission to perform this action.'}, status=status.HTTP_403_FORBIDDEN)

        try:
            logistics_partner = User.objects.get(id=logistics_partner_id, role='logistics')
            order.logistics_partner = logistics_partner
            order.save()
            return Response({'status': f'Order assigned to logistics partner {logistics_partner.username}'}, status=status.HTTP_200_OK)
        except User.DoesNotExist:
            return Response({'detail': 'Logistics partner not found.'}, status=status.HTTP_404_NOT_FOUND)


class OrderItemViewSet(viewsets.ModelViewSet):
    queryset = OrderItem.objects.all()
    serializer_class = OrderItemSerializer
    permission_classes = [IsAuthenticated]

    # Typically, OrderItems are created/managed via the Order creation process.
    # Direct CRUD for OrderItem might be restricted or handled differently.
    # For simplicity, we allow it here but in a real app, you'd refine permissions.
    def get_queryset(self):
        # Only show items for orders the user can see
        if self.request.user.role == 'patient':
            return OrderItem.objects.filter(order__patient=self.request.user)
        elif self.request.user.role == 'pharmacy':
            return OrderItem.objects.filter(order__pharmacy=self.request.user)
        elif self.request.user.role == 'logistics':
            return OrderItem.objects.filter(order__logistics_partner=self.request.user)
        elif self.request.user.is_staff:
            return OrderItem.objects.all()
        return OrderItem.objects.none()

```python
# mediconnect_project/core/integrations.py
# Placeholder functions for third-party integrations.
# In a real application, these would involve actual API calls.

# --- Twilio SMS Integration ---
# from twilio.rest import Client # Uncomment and install twilio if you use this

def send_sms(to_number, message_body):
    """
    Sends an SMS message using Twilio.
    Replace with actual Twilio API calls.
    """
    # TWILIO_ACCOUNT_SID = 'ACxxxxxxxxxxxxxxxxxxxxxxxxxxxxx' # Your Account SID from twilio.com/console
    # TWILIO_AUTH_TOKEN = 'your_auth_token' # Your Auth Token from twilio.com/console
    # YOUR_TWILIO_NUMBER = '+15017122661' # Your Twilio phone number

    print(f"--- Simulating SMS to {to_number}: {message_body} ---")
    # client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)
    # message = client.messages.create(
    #     to=to_number,
    #     from_=YOUR_TWILIO_NUMBER,
    #     body=message_body
    # )
    # print(f"SMS sent with SID: {message.sid}")
    return {"status": "success", "message": "SMS simulated successfully"}

# --- Safaricom M-Pesa Integration ---
# from safaricom.api import Mpesa # Uncomment and install safaricom-python-sdk if you use this

def initiate_payment(phone_number, amount):
    """
    Initiates an M-Pesa STK Push payment.
    Replace with actual M-Pesa API calls.
    """
    # MPESA_CONSUMER_KEY = 'your_consumer_key'
    # MPESA_CONSUMER_SECRET = 'your_consumer_secret'
    # mpesa = Mpesa(consumer_key=MPESA_CONSUMER_KEY, consumer_secret=MPESA_CONSUMER_SECRET)

    print(f"--- Simulating M-Pesa STK Push for {amount} to {phone_number} ---")
    # response = mpesa.stk_push(phone_number, amount, "MediConnect Payment")
    # print(f"M-Pesa response: {response}")
    return {"status": "success", "message": "M-Pesa payment simulated successfully"}

# --- AI/ML (OCR) Integration ---
# import pytesseract # Uncomment and install pytesseract if you use this
# from PIL import Image # Uncomment and install Pillow if you use this

def extract_text_from_image(image_path):
    """
    Extracts text from an image using OCR (e.g., Tesseract).
    Replace with actual OCR model inference.
    """
    print(f"--- Simulating OCR for image: {image_path} ---")
    # try:
    #     text = pytesseract.image_to_string(Image.open(image_path))
    #     return text
    # except Exception as e:
    #     print(f"Error during OCR simulation: {e}")
    #     return "OCR failed or no text found."
    return "Simulated extracted text: Paracetamol 500mg, 2 tablets daily for 5 days."

# --- GIS Integration (Geocoding) ---
# from geopy.geocoders import Nominatim # Uncomment and install geopy if you use this

def geocode_address(address):
    """
    Geocodes an address to latitude and longitude.
    Replace with actual GIS API calls (e.g., Google Maps Geocoding API).
    """
    print(f"--- Simulating geocoding for address: {address} ---")
    # geolocator = Nominatim(user_agent="mediconnect_app")
    # try:
    #     location = geolocator.geocode(address)
    #     if location:
    #         return {"latitude": location.latitude, "longitude": location.longitude}
    #     else:
    #         return None
    # except Exception as e:
    #     print(f"Error during geocoding simulation: {e}")
    #     return None
    return {"latitude": -1.286389, "longitude": 36.817223} # Example Nairobi coordinates

```python
# mediconnect_project/core/urls.py
# URL routing for the core application using Django REST Framework Routers.

from rest_framework.routers import DefaultRouter
from .views import UserViewSet, PrescriptionViewSet, MedicineViewSet, OrderViewSet, OrderItemViewSet

router = DefaultRouter()
router.register(r'users', UserViewSet)
router.register(r'prescriptions', PrescriptionViewSet)
router.register(r'medicines', MedicineViewSet)
router.register(r'orders', OrderViewSet)
router.register(r'orderitems', OrderItemViewSet)

urlpatterns = router.urls

```python
# mediconnect_project/core/admin.py
# Register models with the Django admin interface.

from django.contrib import admin
from django.contrib.auth.admin import UserAdmin
from .models import User, Prescription, Medicine, Order, OrderItem

# Register our custom User model with the admin site
@admin.register(User)
class CustomUserAdmin(UserAdmin):
    # Add 'phone_number' and 'role' to the fields displayed in the admin list and forms
    fieldsets = UserAdmin.fieldsets + (
        (None, {'fields': ('phone_number', 'role')}),
    )
    add_fieldsets = UserAdmin.add_fieldsets + (
        (None, {'fields': ('phone_number', 'role')}),
    )
    list_display = UserAdmin.list_display + ('phone_number', 'role')

# Register other models
admin.site.register(Prescription)
admin.site.register(Medicine)
admin.site.register(Order)
admin.site.register(OrderItem)

```python
# mediconnect_project/mediconnect_project/asgi.py
# ASGI config for mediconnect_project project.
# It exposes the ASGI callable as a module-level variable named ``application``.
# For more information on this file, see
# [https://docs.djangoproject.com/en/5.0/howto/deployment/asgi/](https://docs.djangoproject.com/en/5.0/howto/deployment/asgi/)

import os

from django.core.asgi import get_asgi_application

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'mediconnect_project.settings')

application = get_asgi_application()

```python
# mediconnect_project/mediconnect_project/wsgi.py
# WSGI config for mediconnect_project project.
# It exposes the WSGI callable as a module-level variable named ``application``.
# For more information on this file, see
# https://docs.djangoproject.com/en/5.0/howto/deployment/wsgi/

import os

from django.core.wsgi import get_wsgi_application

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'mediconnect_project.settings')

application = get_wsgi_application()